# Structured Streaming using the Python DataFrames API

Apache Spark includes a high-level stream processing API, [Structured Streaming](http://spark.apache.org/docs/latest/structured-streaming-programming-guide.html). In this notebook we take a quick look at how to use the DataFrame API to build Structured Streaming applications. We want to compute real-time metrics like running counts and windowed counts on a stream of timestamped actions (e.g. Open, Close, etc).

To run this notebook, import it and attach it to a Spark cluster.

## Sample Data
We have some sample action data as files in `/databricks-datasets/structured-streaming/events/` which we are going to use to build this appication. Let's take a look at the contents of this directory.

In [0]:
%fs head /databricks-datasets/structured-streaming/events/file-1.json

[Truncated to first 65536 bytes]
{"time":1469506633,"action":"Close"}
{"time":1469506636,"action":"Close"}
{"time":1469506642,"action":"Open"}
{"time":1469506644,"action":"Open"}
{"time":1469506646,"action":"Open"}
{"time":1469506647,"action":"Open"}
{"time":1469506648,"action":"Open"}
{"time":1469506651,"action":"Open"}
{"time":1469506653,"action":"Close"}
{"time":1469506653,"action":"Open"}
{"time":1469506656,"action":"Open"}
{"time":1469506659,"action":"Open"}
{"time":1469506659,"action":"Open"}
{"time":1469506660,"action":"Close"}
{"time":1469506660,"action":"Close"}
{"time":1469506662,"action":"Open"}
{"time":1469506668,"action":"Close"}
{"time":1469506669,"action":"Close"}
{"time":1469506670,"action":"Open"}
{"time":1469506670,"action":"Open"}
{"time":1469506675,"action":"Close"}
{"time":1469506675,"action":"Open"}
{"time":1469506679,"action":"Open"}
{"time":1469506685,"action":"Close"}
{"time":1469506685,"action":"Close"}
{"time":1469506687,"action":"Open"}
{"time":1469506691,"action":"Close"}
{"time":1469506700,"action":"Close"}
{"time":1469506705,"action":"Close"}
{"time":1469506707,"action":"Open"}
{"time":1469506707,"action":"Open"}
{"time":1469506708,"action":"Close"}
{"time":1469506715,"action":"Close"}
{"time":1469506721,"action":"Close"}
{"time":1469506721,"action":"Open"}
{"time":1469506722,"action":"Open"}
{"time":1469506723,"action":"Open"}
{"time":1469506727,"action":"Open"}
{"time":1469506729,"action":"Close"}
{"time":1469506729,"action":"Close"}
{"time":1469506729,"action":"Open"}
{"time":1469506731,"action":"Open"}
{"time":1469506732,"action":"Close"}
{"time":1469506739,"action":"Close"}
{"time":1469506739,"action":"Close"}
{"time":1469506740,"action":"Close"}
{"time":1469506740,"action":"Open"}
{"time":1469506741,"action":"Close"}
{"time":1469506743,"action":"Close"}
{"time":1469506749,"action":"Open"}
{"time":1469506757,"action":"Close"}
{"time":1469506759,"action":"Open"}
{"time":1469506760,"action":"Open"}
{"time":1469506760,"action":"Open"}
{"time":1469506761,"action":"Open"}
{"time":1469506765,"action":"Open"}
{"time":1469506768,"action":"Close"}
{"time":1469506769,"action":"Close"}
{"time":1469506771,"action":"Open"}
{"time":1469506781,"action":"Close"}
{"time":1469506782,"action":"Open"}
{"time":1469506784,"action":"Open"}
{"time":1469506790,"action":"Open"}
{"time":1469506794,"action":"Open"}
{"time":1469506795,"action":"Close"}
{"time":1469506796,"action":"Open"}
{"time":1469506797,"action":"Open"}
{"time":1469506797,"action":"Open"}
{"time":1469506798,"action":"Open"}
{"time":1469506800,"action":"Open"}
{"time":1469506803,"action":"Open"}
{"time":1469506803,"action":"Open"}
{"time":1469506805,"action":"Close"}
{"time":1469506806,"action":"Open"}
{"time":1469506807,"action":"Open"}
{"time":1469506810,"action":"Close"}
{"time":1469506815,"action":"Open"}
{"time":1469506816,"action":"Open"}
{"time":1469506816,"action":"Open"}
{"time":1469506819,"action":"Open"}
{"time":1469506821,"action":"Open"}
{"time":1469506824,"action":"Close"}
{"time":1469506828,"action":"Close"}
{"time":1469506829,"action":"Close"}
{"time":1469506831,"action":"Close"}
{"time":1469506836,"action":"Close"}
{"time":1469506838,"action":"Close"}
{"time":1469506840,"action":"Open"}
{"time":1469506842,"action":"Close"}
{"time":1469506842,"action":"Open"}
{"time":1469506843,"action":"Open"}
{"time":1469506846,"action":"Open"}
{"time":1469506848,"action":"Close"}
{"time":1469506849,"action":"Close"}
{"time":1469506854,"action":"Open"}
{"time":1469506854,"action":"Open"}
{"time":1469506856,"action":"Close"}
{"time":1469506858,"action":"Close"}
{"time":1469506859,"action":"Close"}
{"time":1469506859,"action":"Close"}
{"time":1469506860,"action":"Open"}
{"time":1469506863,"action":"Open"}
{"time":1469506865,"action":"Close"}
{"time":1469506869,"action":"Open"}
{"time":1469506869,"action":"Open"}
{"time":1469506870,"action":"Open"}
{"time":1469506872,"action":"Close"}
{"time":1469506872,"action":"Open"}
{"time":1469506873,"action":"Open

In [0]:
#%fs mkdirs /FileStore/streaming
#%fs cp /databricks-datasets/structured-streaming/events/file-1.json /FileStore/streaming
#%fs cp /FileStore/tables/file_2.json /FileStore/streaming/
#%fs ls /FileStore/streaming
#%fs head /FileStore/streaming/file_2.json

In [0]:
%fs ls /FileStore/streaming/

path,name,size
dbfs:/FileStore/streaming/file-0.json,file-0.json,72530
dbfs:/FileStore/streaming/file-1.json,file-1.json,72961


In [0]:
%fs ls /databricks-datasets/structured-streaming/events/

path,name,size
dbfs:/databricks-datasets/structured-streaming/events/file-0.json,file-0.json,72530
dbfs:/databricks-datasets/structured-streaming/events/file-1.json,file-1.json,72961
dbfs:/databricks-datasets/structured-streaming/events/file-10.json,file-10.json,73025
dbfs:/databricks-datasets/structured-streaming/events/file-11.json,file-11.json,72999
dbfs:/databricks-datasets/structured-streaming/events/file-12.json,file-12.json,72987
dbfs:/databricks-datasets/structured-streaming/events/file-13.json,file-13.json,73006
dbfs:/databricks-datasets/structured-streaming/events/file-14.json,file-14.json,73003
dbfs:/databricks-datasets/structured-streaming/events/file-15.json,file-15.json,73007
dbfs:/databricks-datasets/structured-streaming/events/file-16.json,file-16.json,72978
dbfs:/databricks-datasets/structured-streaming/events/file-17.json,file-17.json,73008


There are about 50 JSON files in the directory. Let's see what each JSON file contains.

In [0]:
%fs head /databricks-datasets/structured-streaming/events/file-0.json

[Truncated to first 65536 bytes]
{"time":1469501107,"action":"Open"}
{"time":1469501147,"action":"Open"}
{"time":1469501202,"action":"Open"}
{"time":1469501219,"action":"Open"}
{"time":1469501225,"action":"Open"}
{"time":1469501234,"action":"Open"}
{"time":1469501245,"action":"Open"}
{"time":1469501246,"action":"Open"}
{"time":1469501248,"action":"Open"}
{"time":1469501256,"action":"Open"}
{"time":1469501264,"action":"Open"}
{"time":1469501266,"action":"Open"}
{"time":1469501267,"action":"Open"}
{"time":1469501269,"action":"Open"}
{"time":1469501271,"action":"Open"}
{"time":1469501282,"action":"Open"}
{"time":1469501285,"action":"Open"}
{"time":1469501291,"action":"Open"}
{"time":1469501297,"action":"Open"}
{"time":1469501303,"action":"Open"}
{"time":1469501322,"action":"Open"}
{"time":1469501335,"action":"Open"}
{"time":1469501344,"action":"Open"}
{"time":1469501346,"action":"Open"}
{"time":1469501349,"action":"Open"}
{"time":1469501357,"action":"Open"}
{"time":1469501366,"action":"Open"}
{"time":1469501371,"action":"Open"}
{"time":1469501375,"action":"Open"}
{"time":1469501375,"action":"Open"}
{"time":1469501381,"action":"Open"}
{"time":1469501392,"action":"Open"}
{"time":1469501402,"action":"Open"}
{"time":1469501407,"action":"Open"}
{"time":1469501410,"action":"Open"}
{"time":1469501420,"action":"Open"}
{"time":1469501424,"action":"Open"}
{"time":1469501438,"action":"Open"}
{"time":1469501442,"action":"Close"}
{"time":1469501462,"action":"Open"}
{"time":1469501480,"action":"Open"}
{"time":1469501488,"action":"Open"}
{"time":1469501489,"action":"Open"}
{"time":1469501491,"action":"Open"}
{"time":1469501503,"action":"Open"}
{"time":1469501505,"action":"Open"}
{"time":1469501509,"action":"Open"}
{"time":1469501513,"action":"Open"}
{"time":1469501517,"action":"Open"}
{"time":1469501520,"action":"Open"}
{"time":1469501525,"action":"Open"}
{"time":1469501533,"action":"Open"}
{"time":1469501539,"action":"Open"}
{"time":1469501540,"action":"Open"}
{"time":1469501541,"action":"Open"}
{"time":1469501543,"action":"Open"}
{"time":1469501544,"action":"Open"}
{"time":1469501545,"action":"Close"}
{"time":1469501545,"action":"Open"}
{"time":1469501547,"action":"Open"}
{"time":1469501552,"action":"Open"}
{"time":1469501557,"action":"Open"}
{"time":1469501559,"action":"Open"}
{"time":1469501560,"action":"Open"}
{"time":1469501560,"action":"Open"}
{"time":1469501565,"action":"Open"}
{"time":1469501566,"action":"Open"}
{"time":1469501574,"action":"Open"}
{"time":1469501575,"action":"Open"}
{"time":1469501575,"action":"Open"}
{"time":1469501578,"action":"Open"}
{"time":1469501581,"action":"Open"}
{"time":1469501584,"action":"Open"}
{"time":1469501600,"action":"Open"}
{"time":1469501601,"action":"Open"}
{"time":1469501603,"action":"Open"}
{"time":1469501610,"action":"Open"}
{"time":1469501620,"action":"Open"}
{"time":1469501621,"action":"Open"}
{"time":1469501625,"action":"Open"}
{"time":1469501625,"action":"Close"}
{"time":1469501626,"action":"Open"}
{"time":1469501631,"action":"Open"}
{"time":1469501632,"action":"Open"}
{"time":1469501632,"action":"Open"}
{"time":1469501638,"action":"Open"}
{"time":1469501643,"action":"Open"}
{"time":1469501646,"action":"Open"}
{"time":1469501662,"action":"Open"}
{"time":1469501662,"action":"Open"}
{"time":1469501662,"action":"Open"}
{"time":1469501663,"action":"Open"}
{"time":1469501667,"action":"Open"}
{"time":1469501674,"action":"Open"}
{"time":1469501675,"action":"Open"}
{"time":1469501678,"action":"Close"}
{"time":1469501680,"action":"Open"}
{"time":1469501685,"action":"Open"}
{"time":1469501686,"action":"Open"}
{"time":1469501689,"action":"Open"}
{"time":1469501691,"action":"Open"}
{"time":1469501694,"action":"Open"}
{"time":1469501696,"action":"Close"}
{"time":1469501702,"action":"Open"}
{"time":1469501703,"action":"Open"}
{"time":1469501704,"action":"Open"}
{"time":1469501706,"action":"Open"}
{"time":1469501706,"action":"Open"}
{"time":1469501710,"action":"Open"}
{"time":1469501715,"action":"Open"}
{"

Each line in the file contains JSON record with two fields - `time` and `action`. Let's try to analyze these files interactively.

## Batch/Interactive Processing
The usual first step in attempting to process the data is to interactively query the data. Let's define a static DataFrame on the files, and give it a table name.

In [0]:
from pyspark.sql.types import *

#inputPath = "/databricks-datasets/structured-streaming/events/"
inputPath = "/FileStore/streaming"

# Since we know the data format already, let's define the schema to speed up processing (no need for Spark to infer schema)
jsonSchema = StructType([ StructField("time", TimestampType(), True), StructField("action", StringType(), True) ])

# Static DataFrame representing data in the JSON files
staticInputDF = (
  spark
    .read
    .schema(jsonSchema)
    .json(inputPath)
)

display(staticInputDF)

time,action
2016-07-26T04:17:13.000+0000,Close
2016-07-26T04:17:16.000+0000,Close
2016-07-26T04:17:22.000+0000,Open
2016-07-26T04:17:24.000+0000,Open
2016-07-26T04:17:26.000+0000,Open
2016-07-26T04:17:27.000+0000,Open
2016-07-26T04:17:28.000+0000,Open
2016-07-26T04:17:31.000+0000,Open
2016-07-26T04:17:33.000+0000,Close
2016-07-26T04:17:33.000+0000,Open


Now we can compute the number of "open" and "close" actions with one hour windows. To do this, we will group by the `action` column and 1 hour windows over the `time` column.

In [0]:
from pyspark.sql.functions import *      # for window() function

staticCountsDF = (
  staticInputDF
    .groupBy(
       staticInputDF.action, 
       window(staticInputDF.time, "1 hour"))    
    .count()
)
staticCountsDF.cache()

# Register the DataFrame as table 'static_counts'
staticCountsDF.createOrReplaceTempView("static_counts")

Now we can directly use SQL to query the table. For example, here are the total counts across all the hours.

In [0]:
%sql select action, sum(count) as total_count from static_counts group by action

action,total_count
Open,2507
Close,1493


How about a timeline of windowed counts?

In [0]:
%sql select action, date_format(window.end, "MMM-dd HH:mm") as time, count from static_counts order by time, action

action,time,count
Close,Jul-26 03:00,11
Open,Jul-26 03:00,179
Close,Jul-26 04:00,344
Open,Jul-26 04:00,1001
Close,Jul-26 05:00,815
Open,Jul-26 05:00,999
Close,Jul-26 06:00,323
Open,Jul-26 06:00,328


Note the two ends of the graph. The close actions are generated such that they are after the corresponding open actions, so there are more "opens" in the beginning and more "closes" in the end.

## Stream Processing 
Now that we have analyzed the data interactively, let's convert this to a streaming query that continuously updates as data comes. Since we just have a static set of files, we are going to emulate a stream from them by reading one file at a time, in the chronological order they were created. The query we have to write is pretty much the same as the interactive query above.

In [0]:
from pyspark.sql.functions import *

# Similar to definition of staticInputDF above, just using `readStream` instead of `read`
streamingInputDF = (
  spark
    .readStream                       
    .schema(jsonSchema)               # Set the schema of the JSON data
    .option("maxFilesPerTrigger", 1)  # Treat a sequence of files as a stream by picking one file at a time
    .json(inputPath)
)

# Same query as staticInputDF
streamingCountsDF = (                 
  streamingInputDF
    .groupBy(
      streamingInputDF.action, 
      window(streamingInputDF.time, "1 hour"))
    .count()
)

# Is this DF actually a streaming DF?
streamingCountsDF.isStreaming

Out[47]: True

As you can see, `streamingCountsDF` is a streaming Dataframe (`streamingCountsDF.isStreaming` was `true`). You can start streaming computation, by defining the sink and starting it. 
In our case, we want to interactively query the counts (same queries as above), so we will set the complete set of 1 hour counts to be in a in-memory table.

In [0]:
spark.conf.set("spark.sql.shuffle.partitions", "2")  # keep the size of shuffles small

query = (
  streamingCountsDF
    .writeStream
    .format("memory")        # memory = store in-memory table 
    .queryName("counts")     # counts = name of the in-memory table
    .outputMode("complete")  # complete = all the counts should be in the table
    .start()
)

`query` is a handle to the streaming query that is running in the background. This query is continuously picking up files and updating the windowed counts. 

Note the status of query in the above cell. The progress bar shows that the query is active. 
Furthermore, if you expand the `> counts` above, you will find the number of files they have already processed. 

Let's wait a bit for a few files to be processed and then interactively query the in-memory `counts` table.

In [0]:
from time import sleep
sleep(5)  # wait a bit for computation to start

In [0]:
%sql select action, date_format(window.end, "MMM-dd HH:mm") as time, count from counts order by time, action

action,time,count
Close,Jul-26 03:00,11
Open,Jul-26 03:00,179
Close,Jul-26 04:00,344
Open,Jul-26 04:00,1001
Close,Jul-26 05:00,176
Open,Jul-26 05:00,289


We see the timeline of windowed counts (similar to the static one earlier) building up. If we keep running this interactive query repeatedly, we will see the latest updated counts which the streaming query is updating in the background.

In [0]:
sleep(5)  # wait a bit more for more data to be computed

In [0]:
%sql select action, date_format(window.end, "MMM-dd HH:mm") as time, count from counts order by time, action

action,time,count
Close,Jul-26 03:00,11
Open,Jul-26 03:00,179
Close,Jul-26 04:00,344
Open,Jul-26 04:00,1001
Close,Jul-26 05:00,815
Open,Jul-26 05:00,999
Close,Jul-26 06:00,323
Open,Jul-26 06:00,328


In [0]:
sleep(5)  # wait a bit more for more data to be computed

In [0]:
#%fs cp /FileStore/tables/file_2.json /FileStore/streaming/

res49: Boolean = true

In [0]:
%sql select action, date_format(window.end, "MMM-dd HH:mm") as time, count from counts order by time, action

action,time,count
Close,Jul-26 03:00,11
Open,Jul-26 03:00,180
Close,Jul-26 04:00,344
Open,Jul-26 04:00,1001
Close,Jul-26 05:00,815
Open,Jul-26 05:00,1000
Close,Jul-26 06:00,323
Open,Jul-26 06:00,328


Also, let's see the total number of "opens" and "closes".

In [0]:
%sql select action, sum(count) as total_count from counts group by action order by action

action,total_count
Close,1493
Open,2509


If you keep running the above query repeatedly, you will always find that the number of "opens" is more than the number of "closes", as expected in a data stream where a "close" always appear after corresponding "open". This shows that Structured Streaming ensures **prefix integrity**. Read the blog posts linked below if you want to know more.

Note that there are only a few files, so consuming all of them there will be no updates to the counts. Rerun the query if you want to interact with the streaming query again.

Finally, you can stop the query running in the background, either by clicking on the 'Cancel' link in the cell of the query, or by executing `query.stop()`. Either way, when the query is stopped, the status of the corresponding cell above will automatically update to `TERMINATED`.